# Temporary Files and Folders with `tempfile`

In this notebook we explore creating temporary files and directories, controlling names (suffix/prefix), deciding whether they persist after context exit, and choosing where they are placed.

In [2]:
import tempfile
import os
from pathlib import Path
import json
import csv

## Default Temporary Location
Check where Python places temp resources by default.

In [3]:
default_tmp = tempfile.gettempdir()
print(f"Default temp dir: {default_tmp}")

# Show a sample temp dir then remove it
sample = tempfile.mkdtemp()
print(f"Sample created: {sample}")
os.rmdir(sample)
print(f"Exists after cleanup? {os.path.exists(sample)}")

Default temp dir: C:\Users\zepedro\AppData\Local\Temp
Sample created: C:\Users\zepedro\AppData\Local\Temp\tmpl2ci1tkj
Exists after cleanup? False


## 1) Temporary Files as Contexts (auto-delete)
`TemporaryFile` and `NamedTemporaryFile` clean up on exit when `delete=True` (default for named).

In [4]:
# Anonymous temp file (not visible on disk name)
with tempfile.TemporaryFile(mode="w+") as f:
    f.write("hello temp\n")
    f.seek(0)
    print(f"Anonymous content: {f.read().strip()}")
print("Anonymous temp auto-removed")

# Named temp file with suffix (visible on disk) auto-deletes
with tempfile.NamedTemporaryFile(mode="w+", suffix=".txt") as f:
    print(f"Named temp path: {f.name}")
    f.write("named temp data")
    f.flush()
    f.seek(0)
    print(f"Named content: {f.read()}")
print(f"Exists after context? {os.path.exists(f.name)}")

Anonymous content: hello temp
Anonymous temp auto-removed
Named temp path: C:\Users\zepedro\AppData\Local\Temp\tmpkl1gs_w9.txt
Named content: named temp data
Exists after context? False


## 2) Let Files Persist After Context (`delete=False`)
Use `delete=False` to keep files for later inspection; clean up manually.

In [5]:
with tempfile.NamedTemporaryFile(mode="w", suffix=".log", delete=False) as f:
    f.write("persist me\n")
    persist_path = Path(f.name)
    print(f"Created persistent file: {persist_path}")

print(f"Exists after context? {persist_path.exists()}")
print(f"Content: {persist_path.read_text().strip()}")

persist_path.unlink()
print(f"Exists after manual delete? {persist_path.exists()}")

Created persistent file: C:\Users\zepedro\AppData\Local\Temp\tmpk38ry0f5.log
Exists after context? True
Content: persist me
Exists after manual delete? False


## 3) Control Names with Prefix/Suffix
Set identifiable prefixes/suffixes while letting `tempfile` guarantee uniqueness.

In [6]:
paths = []
for prefix, suffix in [("myapp_", ".log"), ("data_", ".csv"), ("cfg_", ".json")]:
    with tempfile.NamedTemporaryFile(prefix=prefix, suffix=suffix, delete=False) as f:
        print(f"Created: {Path(f.name).name}")
        paths.append(Path(f.name))

print("\nCleanup")
for p in paths:
    p.unlink()
    print(f"Deleted {p.name}")

Created: myapp_n9gbd4di.log
Created: data_esxag72m.csv
Created: cfg_5viioxfz.json

Cleanup
Deleted myapp_n9gbd4di.log
Deleted data_esxag72m.csv
Deleted cfg_5viioxfz.json


## 4) Control Location with `dir`
Place temp files in a specific folder, and compare against the default location.

In [7]:
custom_dir = Path.cwd() / "custom_tmp"
custom_dir.mkdir(exist_ok=True)

with tempfile.NamedTemporaryFile(dir=custom_dir, suffix=".txt", delete=False) as f_custom:
    f_custom.write(b"custom location")
    custom_path = Path(f_custom.name)
    print(f"Custom file: {custom_path}")

with tempfile.NamedTemporaryFile(suffix=".txt", delete=False) as f_default:
    default_path = Path(f_default.name)
    print(f"Default-located file: {default_path}")

print(f"Custom in dir? {custom_path.parent == custom_dir}")
print(f"Default in temp dir? {default_path.parent == Path(tempfile.gettempdir())}")

# Clean up
custom_path.unlink()
default_path.unlink()
custom_dir.rmdir()
print("Cleaned custom dir and files")

Custom file: c:\Users\zepedro\Documents\GitHub\PMNEC\custom_tmp\tmpsqmm5bvb.txt
Default-located file: C:\Users\zepedro\AppData\Local\Temp\tmptu1ka8rg.txt
Custom in dir? True
Default in temp dir? True
Cleaned custom dir and files


## 5) Temporary Directories
`TemporaryDirectory` removes itself (and contents) on exit; use `delete=False` behavior by manually managing if needed.

In [8]:
# Auto-cleaning temporary directory
with tempfile.TemporaryDirectory(prefix="example_") as d:
    dpath = Path(d)
    (dpath / "data.txt").write_text("some data")
    (dpath / "config.ini").write_text("key=value")
    print(f"Temp dir: {dpath}")
    print("Contents:", [p.name for p in dpath.iterdir()])
print(f"Exists after context? {dpath.exists()}")

Temp dir: C:\Users\zepedro\AppData\Local\Temp\example_qq3p3tjx
Contents: ['config.ini', 'data.txt']
Exists after context? False


### Persistent temp directory (manual cleanup)
Use `TemporaryDirectory` and move/rename before exit, or create with `mkdtemp`.

In [9]:
# Persist by creating with mkdtemp (no auto-delete)
persist_dir = Path(tempfile.mkdtemp(prefix="persist_"))
print(f"Persistent dir: {persist_dir}")
(persist_dir / "keep.txt").write_text("keep this")
print("Exists after creation?", persist_dir.exists())
print("Files:", [p.name for p in persist_dir.iterdir()])

# Manual cleanup
for p in persist_dir.iterdir():
    p.unlink()
persist_dir.rmdir()
print("Persistent dir removed manually")

Persistent dir: C:\Users\zepedro\AppData\Local\Temp\persist_4mkbtmmt
Exists after creation? True
Files: ['keep.txt']
Persistent dir removed manually


## Summary
- Use contexts (`TemporaryFile`, `NamedTemporaryFile`, `TemporaryDirectory`) for auto-cleanup.
- Use `suffix`/`prefix` to tag temp resources.
- Set `delete=False` (or `mkdtemp`) when you need to keep things.
- Use `dir` to choose location; default from `tempfile.gettempdir()`.